In [1]:
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random, os, pathlib

import torch
import torch.nn as nn
from torch.utils import data

## MNIST dataset

In [2]:
import mylibrary.datasets as datasets
import mylibrary.nnlib as tnn
from classes import *

In [3]:
mnist = datasets.MNIST()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

# train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [4]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [5]:
def shuffle_data():
    global train_data, train_label
    randidx = random.sample(range(len(train_label)), k=len(train_label))
    train_data = train_data[randidx]
    train_label = train_label[randidx]

In [6]:
input_size = 784
output_size = 10

learning_rate = 0.0001
batch_size = 50

In [7]:
network_seeds = [147, 258, 369]
# network_seeds = [369]
network_seed = 369

EPOCHS = 20

actf = nn.LeakyReLU
# actf = nn.ELU

learning_rate = 0.005
lambda_ = 2
criterion = nn.BCELoss()
sigmoid = nn.Sigmoid()

use_mixup = True

In [8]:
class MNIST_OneClass_Balanced(data.Dataset):
    
    def __init__(self, data, label, class_index):
        self.data = data
        self.label = label
        self.class_index = class_index
        
        mask = (label==class_index)
        self.label = mask.type(torch.float32).reshape(-1,1)
        self.class_data = torch.nonzero(mask).reshape(-1)
        self.other_data = torch.nonzero(~mask).reshape(-1)
        
        random.seed(network_seed)
        self._shuffle_data_()
        self.count = 0
        
    def __len__(self):
        return 2*len(self.class_data)
    
    def _shuffle_data_(self):
#         randidx = np.random.permutation(len(self.other_data))
        randidx = random.sample(range(len(self.other_data)), k=len(self.other_data))
        self.other_data = self.other_data[randidx]
    
    def __getitem__(self, idx):
        if idx < len(self.class_data):
            idx = self.class_data[idx]
            img, lbl = self.data[idx], self.label[idx]
        else:
            idx = self.other_data[idx-len(self.class_data)]
            img, lbl = self.data[idx], self.label[idx]
            self.count += 1
            if self.count >= len(self.class_data): 
                self._shuffle_data_()
                self.count = 0
        return img, lbl

In [9]:
# class_idx = 0
# train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
# test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

In [10]:
# len(train_dataset), len(test_dataset)

In [11]:
# train_loader_all = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
# test_loader_all = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [12]:
# %matplotlib inline
# # img, lbl = train_dataset[11010]
# img, lbl = test_dataset[10]
# print(lbl)
# plt.imshow(img.reshape(28,28))

## Logistic Regression

In [13]:
# class_idx = 0
stat_per_class = []
net_list = []
for class_idx in range(10):
    train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
    test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

    torch.manual_seed(network_seed)
    logNet = LogisticRegression(784)
    optimizer = torch.optim.Adam(logNet.parameters(), lr=learning_rate)

    losses = []
    train_accs = []
    test_accs = []

    index = 0
    for epoch in range(20):
        train_acc = 0
        train_count = 0
        for xx, yy in train_loader:
            index += 1

            if use_mixup:
                rand_indx = np.random.permutation(len(xx))
                rand_lambda = 1-torch.rand(len(xx), 1)*0.1
                x_mix = rand_lambda*xx+(1-rand_lambda)*xx[rand_indx]
                y_mix = rand_lambda*yy+(1-rand_lambda)*yy[rand_indx]
            else:
                x_mix = xx
                y_mix = yy

            yout = sigmoid(logNet(x_mix))    
            loss = criterion(yout, y_mix)
            losses.append(float(loss))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            preds = (yy.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == preds).astype(float).sum()
            train_acc += correct
            train_count += len(outputs)

            if index%200 == 0:
                train_accs.append(float(train_acc)/train_count*100)
                train_acc = 0
                train_count = 0

                print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
                test_count = 0
                test_acc = 0
                for xx, yy in test_loader:
                    with torch.no_grad():
                        yout = logNet(xx)
                    outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
                    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
                    test_acc += correct
                    test_count += len(xx)
                test_accs.append(float(test_acc)/test_count*100)
                print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
                print("\n")
                
    ### after each class index is finished training
    stat_per_class.append(
    f'Class: {class_idx} -> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}'
    )
    print(stat_per_class[-1], '\n')
    net_list.append(logNet)

Epoch: 0:200,  Loss:0.5145545601844788
Train Acc:50.19%, Test Acc:97.86%


Epoch: 1:400,  Loss:0.5180794596672058
Train Acc:50.23%, Test Acc:98.37%


Epoch: 2:600,  Loss:0.49276816844940186
Train Acc:50.68%, Test Acc:98.67%


Epoch: 3:800,  Loss:0.47931113839149475
Train Acc:51.96%, Test Acc:98.62%


Epoch: 4:1000,  Loss:0.5393970608711243
Train Acc:52.35%, Test Acc:98.62%


Epoch: 5:1200,  Loss:0.5352065563201904
Train Acc:55.47%, Test Acc:98.67%


Epoch: 5:1400,  Loss:0.5227574110031128
Train Acc:54.43%, Test Acc:98.67%


Epoch: 6:1600,  Loss:0.5396431684494019
Train Acc:55.88%, Test Acc:98.67%


Epoch: 7:1800,  Loss:0.5722149610519409
Train Acc:58.11%, Test Acc:98.72%


Epoch: 8:2000,  Loss:0.5416309237480164
Train Acc:59.77%, Test Acc:98.72%


Epoch: 9:2200,  Loss:0.5448659658432007
Train Acc:61.73%, Test Acc:98.72%


Epoch: 10:2400,  Loss:0.47605592012405396
Train Acc:64.13%, Test Acc:98.83%


Epoch: 10:2600,  Loss:0.5507673025131226
Train Acc:62.40%, Test Acc:98.83%


Epoch: 11:2

Train Acc:61.60%, Test Acc:96.38%


Epoch: 7:1800,  Loss:0.5752459764480591
Train Acc:63.23%, Test Acc:96.28%


Epoch: 8:2000,  Loss:0.48756495118141174
Train Acc:64.34%, Test Acc:96.44%


Epoch: 9:2200,  Loss:0.5136403441429138
Train Acc:65.23%, Test Acc:96.38%


Epoch: 10:2400,  Loss:0.5440595149993896
Train Acc:67.37%, Test Acc:96.54%


Epoch: 11:2600,  Loss:0.5845538973808289
Train Acc:68.62%, Test Acc:96.64%


Epoch: 11:2800,  Loss:0.5023119449615479
Train Acc:68.25%, Test Acc:96.69%


Epoch: 12:3000,  Loss:0.4997398257255554
Train Acc:68.49%, Test Acc:96.44%


Epoch: 13:3200,  Loss:0.5358188152313232
Train Acc:69.97%, Test Acc:96.54%


Epoch: 14:3400,  Loss:0.5366041660308838
Train Acc:69.98%, Test Acc:96.64%


Epoch: 15:3600,  Loss:0.5245886445045471
Train Acc:71.33%, Test Acc:96.59%


Epoch: 16:3800,  Loss:0.5042750835418701
Train Acc:71.68%, Test Acc:96.59%


Epoch: 17:4000,  Loss:0.47352486848831177
Train Acc:74.09%, Test Acc:96.59%


Epoch: 17:4200,  Loss:0.5423820614814758


Train Acc:60.00%, Test Acc:91.07%


Epoch: 17:4200,  Loss:0.5907708406448364
Train Acc:60.27%, Test Acc:91.12%


Epoch: 18:4400,  Loss:0.527740478515625
Train Acc:60.89%, Test Acc:91.12%


Epoch: 19:4600,  Loss:0.5166277885437012
Train Acc:60.98%, Test Acc:91.22%


Class: 8 -> Train Acc 60.977777777777774 ; Test Acc 91.22176591375771 

Epoch: 0:200,  Loss:0.5678597092628479
Train Acc:50.26%, Test Acc:91.28%


Epoch: 1:400,  Loss:0.523908793926239
Train Acc:51.80%, Test Acc:92.17%


Epoch: 2:600,  Loss:0.5640270113945007
Train Acc:55.66%, Test Acc:92.67%


Epoch: 3:800,  Loss:0.511634111404419
Train Acc:57.33%, Test Acc:93.11%


Epoch: 4:1000,  Loss:0.5540332198143005
Train Acc:59.38%, Test Acc:93.06%


Epoch: 5:1200,  Loss:0.5439659357070923
Train Acc:60.40%, Test Acc:92.96%


Epoch: 5:1400,  Loss:0.5291340351104736
Train Acc:62.60%, Test Acc:93.11%


Epoch: 6:1600,  Loss:0.5902145504951477
Train Acc:63.59%, Test Acc:93.26%


Epoch: 7:1800,  Loss:0.563675582408905
Train Acc:65.00%, Tes

In [14]:
## find the classification error from classification per class network.
acc_test = 0
count_test = 0
with torch.no_grad():
    shuffle_data()
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        yy = test_label[index * batch_size:(index + 1) * batch_size]
        yout = []
        for net in net_list:
            yout.append(net(xx).reshape(-1))
        yout = torch.stack(yout, dim=1).argmax(dim=1)
        
        acc = (yout == yy).type(torch.float32).sum()
        count_test += len(xx)
        acc_test += acc
        
for s in stat_per_class:
    print(s)
print(f"Total Accuracy (Argmax) is : {float(acc_test/count_test)}")

Class: 0 -> Train Acc 72.57731958762886 ; Test Acc 98.87755102040816
Class: 1 -> Train Acc 88.0 ; Test Acc 98.54625550660792
Class: 2 -> Train Acc 68.3061224489796 ; Test Acc 95.44573643410853
Class: 3 -> Train Acc 71.0 ; Test Acc 95.0
Class: 4 -> Train Acc 74.0909090909091 ; Test Acc 96.79226069246437
Class: 5 -> Train Acc 62.0 ; Test Acc 94.05829596412556
Class: 6 -> Train Acc 76.67164179104478 ; Test Acc 97.28601252609603
Class: 7 -> Train Acc 78.25806451612904 ; Test Acc 96.49805447470817
Class: 8 -> Train Acc 60.977777777777774 ; Test Acc 91.22176591375771
Class: 9 -> Train Acc 73.25641025641025 ; Test Acc 93.35976214073341
Total Accuracy (Argmax) is : 0.9154000282287598


## Convex

In [15]:
stat_per_class = []
net_list = []
for class_idx in range(10):
    print(class_idx)
    train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
    test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

    torch.manual_seed(network_seed)
    Net = ConvexNN([784,200,100,1], actf)
    optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)
    losses = []
    train_accs = []
    test_accs = []

    index = 0
    for epoch in range(20):
        train_acc = 0
        train_count = 0
        for xx, yy in train_loader:
            index += 1
        
            if use_mixup:
                rand_indx = np.random.permutation(len(xx))
                rand_lambda = 1-torch.rand(len(xx), 1)*0.1
                x_mix = rand_lambda*xx+(1-rand_lambda)*xx[rand_indx]
                y_mix = rand_lambda*yy+(1-rand_lambda)*yy[rand_indx]
            else:
                x_mix = xx
                y_mix = yy

            yout = sigmoid(Net(x_mix))    
            loss = criterion(yout, y_mix)
            losses.append(float(loss))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            preds = (yy.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == preds).astype(float).sum()
            train_acc += correct
            train_count += len(outputs)


            if index%200 == 0:
                train_accs.append(float(train_acc)/train_count*100)
                train_acc = 0
                train_count = 0

                print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
                test_count = 0
                test_acc = 0
                for xx, yy in test_loader:
                    with torch.no_grad():
                        yout = sigmoid(Net(xx))
                    outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
                    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
                    test_acc += correct
                    test_count += len(xx)
                test_accs.append(float(test_acc)/test_count*100)
                print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
                print("\n")
                
    ### after each class index is finished training
    stat_per_class.append(
    f'Class: {class_idx} -> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}'
    )
    print(stat_per_class[-1], '\n')
    net_list.append(Net)

0
Epoch: 0:200,  Loss:0.1902083456516266
Train Acc:96.30%, Test Acc:98.42%


Epoch: 1:400,  Loss:0.16477316617965698
Train Acc:98.49%, Test Acc:98.47%


Epoch: 2:600,  Loss:0.14492975175380707
Train Acc:98.76%, Test Acc:98.42%


Epoch: 3:800,  Loss:0.11025704443454742
Train Acc:99.30%, Test Acc:99.13%


Epoch: 4:1000,  Loss:0.12073925137519836
Train Acc:99.38%, Test Acc:98.83%


Epoch: 5:1200,  Loss:0.13998454809188843
Train Acc:99.33%, Test Acc:99.34%


Epoch: 5:1400,  Loss:0.14511480927467346
Train Acc:99.54%, Test Acc:99.13%


Epoch: 6:1600,  Loss:0.11947750300168991
Train Acc:99.52%, Test Acc:99.18%


Epoch: 7:1800,  Loss:0.12466155737638474
Train Acc:99.62%, Test Acc:99.59%


Epoch: 8:2000,  Loss:0.13368499279022217
Train Acc:99.81%, Test Acc:99.08%


Epoch: 9:2200,  Loss:0.08891221880912781
Train Acc:99.82%, Test Acc:99.13%


Epoch: 10:2400,  Loss:0.12346048653125763
Train Acc:99.80%, Test Acc:99.18%


Epoch: 10:2600,  Loss:0.12881770730018616
Train Acc:99.83%, Test Acc:99.18%




Train Acc:99.27%, Test Acc:98.42%


Epoch: 6:1600,  Loss:0.1950913816690445
Train Acc:99.45%, Test Acc:98.47%


Epoch: 7:1800,  Loss:0.11146356910467148
Train Acc:99.57%, Test Acc:98.47%


Epoch: 8:2000,  Loss:0.14711014926433563
Train Acc:99.64%, Test Acc:98.57%


Epoch: 9:2200,  Loss:0.113343246281147
Train Acc:99.72%, Test Acc:98.52%


Epoch: 10:2400,  Loss:0.12928034365177155
Train Acc:99.73%, Test Acc:98.37%


Epoch: 11:2600,  Loss:0.10810442268848419
Train Acc:99.77%, Test Acc:98.22%


Epoch: 11:2800,  Loss:0.13461264967918396
Train Acc:99.82%, Test Acc:98.57%


Epoch: 12:3000,  Loss:0.12525875866413116
Train Acc:99.83%, Test Acc:98.37%


Epoch: 13:3200,  Loss:0.14600667357444763
Train Acc:99.77%, Test Acc:98.37%


Epoch: 14:3400,  Loss:0.08760692924261093
Train Acc:99.84%, Test Acc:98.57%


Epoch: 15:3600,  Loss:0.1081419587135315
Train Acc:99.89%, Test Acc:98.42%


Epoch: 16:3800,  Loss:0.1541793942451477
Train Acc:99.75%, Test Acc:98.73%


Epoch: 17:4000,  Loss:0.1098400577902

Train Acc:99.31%, Test Acc:96.30%


Epoch: 16:3800,  Loss:0.13254430890083313
Train Acc:99.30%, Test Acc:96.82%


Epoch: 17:4000,  Loss:0.15160377323627472
Train Acc:99.60%, Test Acc:96.20%


Epoch: 17:4200,  Loss:0.14640316367149353
Train Acc:99.41%, Test Acc:96.61%


Epoch: 18:4400,  Loss:0.13230390846729279
Train Acc:99.58%, Test Acc:96.46%


Epoch: 19:4600,  Loss:0.13300004601478577
Train Acc:99.70%, Test Acc:96.82%


Class: 8 -> Train Acc 99.70370370370371 ; Test Acc 96.81724845995893 

9
Epoch: 0:200,  Loss:0.25558242201805115
Train Acc:89.37%, Test Acc:92.77%


Epoch: 1:400,  Loss:0.3021601736545563
Train Acc:93.90%, Test Acc:93.95%


Epoch: 2:600,  Loss:0.24361030757427216
Train Acc:95.05%, Test Acc:94.90%


Epoch: 3:800,  Loss:0.21518684923648834
Train Acc:95.74%, Test Acc:94.55%


Epoch: 4:1000,  Loss:0.2558193504810333
Train Acc:95.71%, Test Acc:95.14%


Epoch: 5:1200,  Loss:0.21317631006240845
Train Acc:97.00%, Test Acc:95.09%


Epoch: 5:1400,  Loss:0.1474662572145462
Train

In [16]:
## find the classification error from classification per class network.
acc_test = 0
count_test = 0
with torch.no_grad():
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        yy = test_label[index * batch_size:(index + 1) * batch_size]
        yout = []
        for net in net_list:
            yout.append(sigmoid(net(xx)).reshape(-1))
        yout = torch.stack(yout, dim=1).argmax(dim=1)
        acc = (yout == yy).type(torch.float32).sum()
        count_test += len(xx)
        acc_test += acc
        
for s in stat_per_class:
    print(s)
print(f"Total Accuracy (Argmax) is : {float(acc_test/count_test)}")

Class: 0 -> Train Acc 100.0 ; Test Acc 99.59183673469387
Class: 1 -> Train Acc 100.0 ; Test Acc 99.51541850220265
Class: 2 -> Train Acc 99.0 ; Test Acc 98.11046511627907
Class: 3 -> Train Acc 100.0 ; Test Acc 98.46534653465346
Class: 4 -> Train Acc 100.0 ; Test Acc 98.77800407331976
Class: 5 -> Train Acc 99.82978723404256 ; Test Acc 97.92600896860986
Class: 6 -> Train Acc 99.87628865979381 ; Test Acc 98.95615866388309
Class: 7 -> Train Acc 100.0 ; Test Acc 98.44357976653697
Class: 8 -> Train Acc 99.70370370370371 ; Test Acc 96.81724845995893
Class: 9 -> Train Acc 99.05128205128206 ; Test Acc 95.98612487611497
Total Accuracy (Argmax) is : 0.9660000205039978


## NN

In [17]:
stat_per_class = []
net_list = []
for class_idx in range(10):
    print(class_idx)
    train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
    test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

    torch.manual_seed(network_seed)
    Net = nn.Sequential(nn.Linear(784, 200),
                       actf(),
                       nn.Linear(200,100),
                       actf(),
                       nn.Linear(100,1))
    optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)
    losses = []
    train_accs = []
    test_accs = []

    index = 0
    for epoch in range(20):
        train_acc = 0
        train_count = 0
        for xx, yy in train_loader:
            index += 1

            if use_mixup:
                rand_indx = np.random.permutation(len(xx))
                rand_lambda = 1-torch.rand(len(xx), 1)*0.1
                x_mix = rand_lambda*xx+(1-rand_lambda)*xx[rand_indx]
                y_mix = rand_lambda*yy+(1-rand_lambda)*yy[rand_indx]
            else:
                x_mix = xx
                y_mix = yy

            yout = sigmoid(Net(x_mix))    
            loss = criterion(yout, y_mix)
            losses.append(float(loss))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            preds = (yy.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == preds).astype(float).sum()
            train_acc += correct
            train_count += len(outputs)

            if index%200 == 0:
                train_accs.append(float(train_acc)/train_count*100)
                train_acc = 0
                train_count = 0

                print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
                test_count = 0
                test_acc = 0
                for xx, yy in test_loader:
                    with torch.no_grad():
                        yout = sigmoid(Net(xx))
                    outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
                    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
                    test_acc += correct
                    test_count += len(xx)
                test_accs.append(float(test_acc)/test_count*100)
                print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
                print()
                
    ### after each class index is finished training
    stat_per_class.append(
    f'Class: {class_idx} -> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}'
    )
    print(stat_per_class[-1], '\n')
    net_list.append(Net)

0
Epoch: 0:200,  Loss:0.21731796860694885
Train Acc:97.29%, Test Acc:99.29%

Epoch: 1:400,  Loss:0.12151513993740082
Train Acc:98.92%, Test Acc:99.39%

Epoch: 2:600,  Loss:0.11258745193481445
Train Acc:99.44%, Test Acc:99.34%

Epoch: 3:800,  Loss:0.11354289203882217
Train Acc:99.73%, Test Acc:99.29%

Epoch: 4:1000,  Loss:0.1234457939863205
Train Acc:99.73%, Test Acc:99.39%

Epoch: 5:1200,  Loss:0.13944566249847412
Train Acc:99.87%, Test Acc:99.34%

Epoch: 5:1400,  Loss:0.11360175162553787
Train Acc:99.91%, Test Acc:99.39%

Epoch: 6:1600,  Loss:0.18856018781661987
Train Acc:99.89%, Test Acc:99.49%

Epoch: 7:1800,  Loss:0.09785205125808716
Train Acc:100.00%, Test Acc:99.54%

Epoch: 8:2000,  Loss:0.1420046091079712
Train Acc:99.96%, Test Acc:99.44%

Epoch: 9:2200,  Loss:0.09902246296405792
Train Acc:99.94%, Test Acc:99.44%

Epoch: 10:2400,  Loss:0.11073046922683716
Train Acc:99.93%, Test Acc:99.49%

Epoch: 10:2600,  Loss:0.12462783604860306
Train Acc:99.98%, Test Acc:99.44%

Epoch: 11:280

Train Acc:99.76%, Test Acc:98.42%

Epoch: 7:1800,  Loss:0.11900530755519867
Train Acc:99.95%, Test Acc:98.63%

Epoch: 8:2000,  Loss:0.11467210948467255
Train Acc:99.86%, Test Acc:98.57%

Epoch: 9:2200,  Loss:0.10531017929315567
Train Acc:99.66%, Test Acc:98.63%

Epoch: 10:2400,  Loss:0.09614083170890808
Train Acc:99.97%, Test Acc:98.73%

Epoch: 11:2600,  Loss:0.12361001968383789
Train Acc:99.92%, Test Acc:99.13%

Epoch: 11:2800,  Loss:0.10536752641201019
Train Acc:99.88%, Test Acc:98.78%

Epoch: 12:3000,  Loss:0.1102680116891861
Train Acc:99.86%, Test Acc:98.63%

Epoch: 13:3200,  Loss:0.12229945510625839
Train Acc:99.91%, Test Acc:98.83%

Epoch: 14:3400,  Loss:0.1066506952047348
Train Acc:99.89%, Test Acc:98.57%

Epoch: 15:3600,  Loss:0.1212250292301178
Train Acc:99.84%, Test Acc:98.37%

Epoch: 16:3800,  Loss:0.10433226823806763
Train Acc:100.00%, Test Acc:98.78%

Epoch: 17:4000,  Loss:0.12613998353481293
Train Acc:100.00%, Test Acc:98.73%

Epoch: 17:4200,  Loss:0.10448730736970901
Tra

Epoch: 17:4200,  Loss:0.111699178814888
Train Acc:99.85%, Test Acc:97.95%

Epoch: 18:4400,  Loss:0.09885604679584503
Train Acc:99.87%, Test Acc:98.56%

Epoch: 19:4600,  Loss:0.10521125793457031
Train Acc:99.93%, Test Acc:98.61%

Class: 8 -> Train Acc 100.0 ; Test Acc 98.81930184804928 

9
Epoch: 0:200,  Loss:0.16223327815532684
Train Acc:92.94%, Test Acc:96.78%

Epoch: 1:400,  Loss:0.13000811636447906
Train Acc:97.56%, Test Acc:97.08%

Epoch: 2:600,  Loss:0.10350021719932556
Train Acc:98.37%, Test Acc:97.82%

Epoch: 3:800,  Loss:0.17198461294174194
Train Acc:98.88%, Test Acc:97.82%

Epoch: 4:1000,  Loss:0.11707755923271179
Train Acc:99.17%, Test Acc:97.97%

Epoch: 5:1200,  Loss:0.12492135167121887
Train Acc:99.40%, Test Acc:97.92%

Epoch: 5:1400,  Loss:0.14391696453094482
Train Acc:99.33%, Test Acc:97.97%

Epoch: 6:1600,  Loss:0.12057922035455704
Train Acc:99.63%, Test Acc:98.36%

Epoch: 7:1800,  Loss:0.14173611998558044
Train Acc:99.58%, Test Acc:98.27%

Epoch: 8:2000,  Loss:0.0931264

In [18]:
## find the classification error from classification per class network.
acc_test = 0
count_test = 0
with torch.no_grad():
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        yy = test_label[index * batch_size:(index + 1) * batch_size]
        yout = []
        for net in net_list:
            yout.append(sigmoid(net(xx)).reshape(-1))
        yout = torch.stack(yout, dim=1).argmax(dim=1)
        acc = (yout == yy).type(torch.float32).sum()
        count_test += len(xx)
        acc_test += acc
        
for s in stat_per_class:
    print(s)
print(f"Total Accuracy (Argmax) is : {float(acc_test/count_test)}")

Class: 0 -> Train Acc 100.0 ; Test Acc 99.54081632653062
Class: 1 -> Train Acc 100.0 ; Test Acc 99.47136563876651
Class: 2 -> Train Acc 100.0 ; Test Acc 99.03100775193798
Class: 3 -> Train Acc 100.0 ; Test Acc 98.76237623762376
Class: 4 -> Train Acc 100.0 ; Test Acc 99.13441955193483
Class: 5 -> Train Acc 100.0 ; Test Acc 99.04708520179372
Class: 6 -> Train Acc 100.0 ; Test Acc 99.37369519832986
Class: 7 -> Train Acc 100.0 ; Test Acc 98.92996108949417
Class: 8 -> Train Acc 100.0 ; Test Acc 98.81930184804928
Class: 9 -> Train Acc 99.91176470588236 ; Test Acc 98.86025768087215
Total Accuracy (Argmax) is : 0.9782999753952026


# Invex

In [19]:
use_mixup = True
use_check = False
check_every = 2
check_size = 100

m_,s_ = 1, 0

In [20]:
stat_per_class = []
net_list = []
for class_idx in range(10):
    print(class_idx)
    train_dataset = MNIST_OneClass_Balanced(train_data, train_label, class_idx)
    test_dataset = MNIST_OneClass_Balanced(test_data, test_label, class_idx)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

    torch.manual_seed(network_seed)
    lips_net = nn.Sequential(nn.Linear(784, 200),
                       actf(),
                       nn.Linear(200,100),
                       actf(),
                       nn.Linear(100,1))
    Net = BasicInvexNet(784, lips_net, lambda_)
    optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)
    losses = []
    train_accs = []
    test_accs = []

    index = 0
    for epoch in range(20):
        train_acc = 0
        train_count = 0
        for xx, yy in train_loader:
            index += 1
            
            optimizer.zero_grad()
            if use_check and epoch%check_every == 0:
                rand_inp = torch.rand(check_size, 784)*m_+s_
                Net(rand_inp)
                Net.compute_penalty_and_clipper()
                Net.gp.backward(retain_graph=True)
            
            if use_mixup:
                rand_indx = np.random.permutation(len(xx))
                rand_lambda = 1-torch.rand(len(xx), 1)*0.1
                x_mix = rand_lambda*xx+(1-rand_lambda)*xx[rand_indx]
                y_mix = rand_lambda*yy+(1-rand_lambda)*yy[rand_indx]
            else:
                x_mix = xx
                y_mix = yy

            yout = sigmoid(Net(x_mix))   
            Net.compute_penalty_and_clipper()
            loss = criterion(yout, y_mix) + Net.gp
            losses.append(float(loss))

            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            preds = (yy.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == preds).astype(float).sum()
            train_acc += correct
            train_count += len(outputs)

            loss.backward()
            optimizer.step()

#             if index%200 == 0:
        train_accs.append(float(train_acc)/train_count*100)
        train_acc = 0
        train_count = 0

        min_val, gp = float(Net.cond.min()) , float(Net.gp)
        print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}, MinVal:{min_val}, gp: {gp}')
        test_count = 0
        test_acc = 0
        for xx, yy in test_loader:
#                     with torch.no_grad():
            yout = sigmoid(Net(xx))
            outputs = (yout.data.cpu().numpy() > 0.5).astype(float)
            correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
            test_acc += correct
            test_count += len(xx)
        test_accs.append(float(test_acc)/test_count*100)
        print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
        print()
                
    ### after each class index is finished training
    stat_per_class.append(
    f'Class: {class_idx} -> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}'
    )
    print(stat_per_class[-1], '\n')
    net_list.append(Net)

0
Epoch: 0:237,  Loss:0.1692323088645935, MinVal:0.6348922848701477, gp: 7.842011545342031e-13
Train Acc:96.16%, Test Acc:99.23%

Epoch: 1:474,  Loss:0.14545102417469025, MinVal:0.5404513478279114, gp: 5.950114012609475e-11
Train Acc:99.00%, Test Acc:99.34%

Epoch: 2:711,  Loss:0.10647765547037125, MinVal:0.2636297047138214, gp: 2.6961085950460983e-06
Train Acc:99.55%, Test Acc:99.39%

Epoch: 3:948,  Loss:0.11226221919059753, MinVal:0.4234794080257416, gp: 3.3187992354299922e-09
Train Acc:99.64%, Test Acc:99.49%

Epoch: 4:1185,  Loss:0.09751855581998825, MinVal:0.7089232206344604, gp: 3.87337528045574e-14
Train Acc:99.81%, Test Acc:99.49%

Epoch: 5:1422,  Loss:0.11645012348890305, MinVal:0.7324582934379578, gp: 1.8279521573163517e-14
Train Acc:99.87%, Test Acc:99.29%

Epoch: 6:1659,  Loss:0.12763670086860657, MinVal:0.3044567406177521, gp: 3.8711553429493506e-07
Train Acc:99.81%, Test Acc:99.29%

Epoch: 7:1896,  Loss:0.1256728321313858, MinVal:0.30716702342033386, gp: 4.7308208195318e-

Train Acc:97.53%, Test Acc:97.57%

Epoch: 2:738,  Loss:0.17326074838638306, MinVal:0.7949886918067932, gp: 5.109913872109384e-15
Train Acc:98.46%, Test Acc:97.92%

Epoch: 3:984,  Loss:0.1539234220981598, MinVal:0.6979528665542603, gp: 2.1355612861862633e-13
Train Acc:98.70%, Test Acc:97.92%

Epoch: 4:1230,  Loss:0.10120690613985062, MinVal:0.5737210512161255, gp: 3.761188235862356e-11
Train Acc:99.11%, Test Acc:98.32%

Epoch: 5:1476,  Loss:0.10354882478713989, MinVal:1.1201485395431519, gp: 1.0766840539138064e-20
Train Acc:99.20%, Test Acc:98.12%

Epoch: 6:1722,  Loss:0.33571746945381165, MinVal:0.7658966779708862, gp: 2.0583403755850167e-14
Train Acc:99.13%, Test Acc:98.27%

Epoch: 7:1968,  Loss:0.15525995194911957, MinVal:0.8028112053871155, gp: 3.2419660895731048e-15
Train Acc:99.35%, Test Acc:97.62%

Epoch: 8:2214,  Loss:0.1693239063024521, MinVal:0.6658638119697571, gp: 7.704913638530153e-13
Train Acc:99.45%, Test Acc:98.22%

Epoch: 9:2460,  Loss:0.06404782086610794, MinVal:0.6497

Epoch: 3:948,  Loss:0.14914003014564514, MinVal:0.9339919090270996, gp: 1.0007963105508638e-17
Train Acc:99.56%, Test Acc:99.11%

Epoch: 4:1185,  Loss:0.16475002467632294, MinVal:0.6760039329528809, gp: 1.7137922754106166e-13
Train Acc:99.71%, Test Acc:98.96%

Epoch: 5:1422,  Loss:0.11780828982591629, MinVal:0.9539639949798584, gp: 3.4170595358090996e-18
Train Acc:99.78%, Test Acc:99.11%

Epoch: 6:1659,  Loss:0.15116865932941437, MinVal:0.6071357727050781, gp: 2.872603196979462e-12
Train Acc:99.82%, Test Acc:99.11%

Epoch: 7:1896,  Loss:0.1063271313905716, MinVal:0.7239356637001038, gp: 3.132661569927621e-14
Train Acc:99.87%, Test Acc:99.11%

Epoch: 8:2133,  Loss:0.08844780921936035, MinVal:0.5871967673301697, gp: 6.153289683341612e-12
Train Acc:99.86%, Test Acc:99.01%

Epoch: 9:2370,  Loss:0.09612610936164856, MinVal:1.0892083644866943, gp: 1.1722016850167302e-20
Train Acc:99.92%, Test Acc:99.22%

Epoch: 10:2607,  Loss:0.12497411668300629, MinVal:0.7679727673530579, gp: 4.725225812300

Train Acc:98.92%, Test Acc:97.67%

Epoch: 5:1428,  Loss:0.13855057954788208, MinVal:0.4298012852668762, gp: 2.4957966804350917e-09
Train Acc:99.10%, Test Acc:97.22%

Epoch: 6:1666,  Loss:0.09654280543327332, MinVal:0.2932624816894531, gp: 8.647716640552972e-07
Train Acc:99.43%, Test Acc:98.12%

Epoch: 7:1904,  Loss:0.12180519849061966, MinVal:0.35416877269744873, gp: 5.718214879379957e-08
Train Acc:99.58%, Test Acc:97.87%

Epoch: 8:2142,  Loss:0.16496817767620087, MinVal:0.7273879051208496, gp: 1.6435898459290013e-14
Train Acc:99.55%, Test Acc:98.32%

Epoch: 9:2380,  Loss:0.18350648880004883, MinVal:0.51267009973526, gp: 8.824947023144958e-11
Train Acc:99.54%, Test Acc:97.52%

Epoch: 10:2618,  Loss:0.14697112143039703, MinVal:0.64967942237854, gp: 3.761438860036548e-13
Train Acc:99.39%, Test Acc:97.97%

Epoch: 11:2856,  Loss:0.14486636221408844, MinVal:1.1981333494186401, gp: 1.0920546085417867e-22
Train Acc:99.48%, Test Acc:97.82%

Epoch: 12:3094,  Loss:0.10552529245615005, MinVal:0.7

In [21]:
## find the classification error from classification per class network.
acc_test = 0
count_test = 0
with torch.no_grad():
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        yy = test_label[index * batch_size:(index + 1) * batch_size]
        yout = []
        for net in net_list:
            yout.append(sigmoid(net(xx)).reshape(-1))
        yout = torch.stack(yout, dim=1).argmax(dim=1)
        acc = (yout == yy).type(torch.float32).sum()
        count_test += len(xx)
        acc_test += acc
        
for s in stat_per_class:
    print(s)
print(f"Total Accuracy (Argmax) is : {float(acc_test/count_test)}")

Class: 0 -> Train Acc 100.0 ; Test Acc 99.48979591836735
Class: 1 -> Train Acc 99.98516760605162 ; Test Acc 99.51541850220265
Class: 2 -> Train Acc 99.95803961060759 ; Test Acc 99.03100775193798
Class: 3 -> Train Acc 99.94291306475289 ; Test Acc 98.51485148514851
Class: 4 -> Train Acc 99.9743238616912 ; Test Acc 99.03258655804481
Class: 5 -> Train Acc 100.0 ; Test Acc 98.87892376681614
Class: 6 -> Train Acc 99.98310239945928 ; Test Acc 99.21711899791231
Class: 7 -> Train Acc 99.99201915403033 ; Test Acc 98.68677042801556
Class: 8 -> Train Acc 99.94872671338233 ; Test Acc 98.870636550308
Class: 9 -> Train Acc 99.95797613044209 ; Test Acc 98.61248761149653
Total Accuracy (Argmax) is : 0.9753000140190125


In [22]:
## only on training and testing data
for class_idx in range(10):
    correct = 0
    count = 0
    net = net_list[class_idx]
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)
        
    for index in range(len(train_label) // batch_size):
        xx = train_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)

    print(f"Class: {class_idx} -> Correct {correct/count*100:.4f}% on {count} input points")

Class: 0 -> Correct 100.0000% on 70000 input points
Class: 1 -> Correct 99.9971% on 70000 input points
Class: 2 -> Correct 100.0000% on 70000 input points
Class: 3 -> Correct 99.9843% on 70000 input points
Class: 4 -> Correct 99.9971% on 70000 input points
Class: 5 -> Correct 100.0000% on 70000 input points
Class: 6 -> Correct 99.9986% on 70000 input points
Class: 7 -> Correct 100.0000% on 70000 input points
Class: 8 -> Correct 99.9986% on 70000 input points
Class: 9 -> Correct 99.9871% on 70000 input points


In [24]:
## Check the constraint on large number of points, including training and test data.
from tqdm import tqdm
for class_idx in range(10):
    correct = 0
    count = 0
    net = net_list[class_idx]
    for index in range(len(test_label) // batch_size):
        xx = test_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)
        
    for index in range(len(train_label) // batch_size):
        xx = train_data[index * batch_size:(index + 1) * batch_size]
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)

    for i in tqdm(range(20000)):
        xx = torch.rand(batch_size, 784)
        net(xx)
        net.compute_penalty_and_clipper()
        correct += (net.cond>0).type(torch.float32).sum()
        count += len(xx)

    print(f"Class: {class_idx} -> Correct {correct/count*100:.4f}% on {count} input points")

100%|██████████| 20000/20000 [01:03<00:00, 316.50it/s]


Class: 0 -> Correct 100.0000% on 1070000 input points


100%|██████████| 20000/20000 [01:10<00:00, 282.42it/s]


Class: 1 -> Correct 99.9998% on 1070000 input points


100%|██████████| 20000/20000 [01:11<00:00, 278.29it/s]


...

Class: 7 -> Correct 33.2285% on 1070000 input points


100%|██████████| 20000/20000 [01:12<00:00, 277.02it/s]


Class: 8 -> Correct 99.9999% on 1070000 input points


100%|██████████| 20000/20000 [01:09<00:00, 288.99it/s]

Class: 9 -> Correct 99.9985% on 1070000 input points
